In [6]:
import pandas as pd

import numpy as np

from sklearn.ensemble import RandomForestClassifier

from sklearn import neighbors, linear_model, preprocessing, svm

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

SCALER = StandardScaler()

svc = svm.LinearSVC()

KNN = neighbors.KNeighborsClassifier()

logistic = linear_model.LogisticRegression(C=1e5)

forest = RandomForestClassifier(n_estimators=10000,
                                random_state=0,
                                n_jobs=-1)

df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
           'Alcalinity of ash', 'Magnesium', 'Total phenols',
           'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
           'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
           'Proline']

df_wine.columns = columns

df_wine.head()


,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [5]:
types = ['KNN', 'svc', 'log']

def split_test_data(X, y):
    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)

    return X_train, X_test, y_train, y_test


def standardize(X_train, X_test):
    X_train_std = SCALER.fit_transform(X_train)

    X_test_std = SCALER.transform(X_test)

    return X_train_std, X_test_std

def classify(X, y, standardize_data=False):
    x_train, x_test, y_train, y_test = split_test_data(X, y)

    if standardize_data:
        X_train_std, X_test_std = standardize(x_train, x_test)

        x_train = X_train_std

        x_test = X_test_std

    scores = {}

    predictions = {}

    for type_ in types:
        classifier = {'KNN': KNN, 'log': logistic, 'svc': svc}.get(type_, '')

        classifier.fit(x_train, y_train)

        predictions[type_] = classifier.predict(x_test)

        scores[type_] = classifier.score(x_test, y_test)

    forest.fit(x_train, y_train)

    return x_train, x_test, y_train, y_test, scores, predictions, forest.feature_importances_


In [15]:


x = df_wine.iloc[:, 1:].values

y = df_wine.iloc[:, 0].values

a = classify(x, y)

x_train, x_test, y_train, y_test, scores, predictions, feature_importances_ = a





In [17]:
xxx= pd.DataFrame(x_test, columns = columns[1:])

xxx['rating'] = y_test
xxx['predictions_knn'] = predictions['KNN']


xxx['predictions_log'] = predictions['log']

xxx['predictions_svc'] = predictions['svc']
xxx.head()

,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline,rating,predictions_knn,predictions_log,predictions_svc
54,13.74,1.67,2.25,16.4,118,2.60,2.90,0.21,1.62,5.85,0.92,3.20,1060,1,1,1,1
151,12.79,2.67,2.48,22.0,112,1.48,1.36,0.24,1.26,10.80,0.48,1.47,480,3,2,3,3
63,12.37,1.13,2.16,19.0,87,3.50,3.10,0.19,1.87,4.45,1.22,2.87,420,2,2,2,2
55,13.56,1.73,2.46,20.5,116,2.96,2.78,0.20,2.45,6.25,0.98,3.03,1120,1,1,1,1
123,13.05,5.80,2.13,21.5,86,2.62,2.65,0.30,2.01,2.60,0.73,3.10,380,2,2,2,2


In [18]:
accuracy = pd.DataFrame(scores, index=[0])

accuracy

,KNN,log,svc
0,0.722222,0.944444,0.944444
